In [1]:
import sqlite3
from datetime import datetime
from pathlib import Path

import hjson
import pandas as pd
from gf_utils.stc_data import get_stc_data
from event_record import EventRecord

gamedata = get_stc_data(
    ["../GF_Data_Tools/data/ch/stc", "../GF_Data_Tools/data/ch/catchdata"],
    "../GF_Data_Tools/data/ch/asset/table",
)
path = Path("../Elisa/logs/develop_log.db").resolve()
conn = sqlite3.connect(f"file:///{path}?immutable=1", uri=True)

In [11]:

eventrecord = EventRecord("./events.hjson")

In [3]:
equip = pd.read_sql("""
select * from equip
""", conn)

In [12]:
def analyze(df):
    series = dict()
    for formula in filter(lambda x: x["develop_type"]==3,gamedata["recommended_formula"].values()):
        mp, ammo, mre, part = formula["produce_mp"], formula["produce_ammo"], formula["produce_mre"], formula["produce_part"]
        sub_df = df.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
        if len(sub_df)==0:
            continue
        # print(mp, ammo, mre, part)

        prev = {k: [int(i) for i in v.split(",")] for k,v in map(lambda x: x.split(":"), formula["preview"].split("|"))}
        rarity = {k: v for k,v in map(lambda x: x.split(":"),formula["type_rarity"].split(","))}
        eqs = filter(lambda x: gamedata["equip"][x]["rank"]==5,sum([v for k,v in prev.items() if rarity[k]=="A"], start=[]))
        # print({k: gamedata["equip"][k]["name"] for k in eqs})
        cnt = len(sub_df)
        record = {}
        record["total"] = cnt
        record["rank5"] = sum(sub_df["equip_rank"]==5)/cnt
        for eid in eqs:
            record[gamedata["equip"][eid]["name"]] = sum(sub_df["equip_id"]==eid)/cnt
        for k, v in record.items():
            series[(mp,ammo,mre,part,k)] = v
    series = pd.Series(series)
    series.index.set_names(["mp","ammo","mre","part","..."],inplace=True)
    return series
        
df = equip.query("trust==True and item_num==1 and input_level==0")
result = []
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    ret = analyze(sub_df)
    ret.name = name
    result.append(ret)

a = pd.concat(result, axis=1)

In [13]:
a.to_excel("equip.xlsx")

In [16]:
df = equip.query("trust==True and item_num==1 and input_level==0")

In [47]:
def analyze(df):
    series = dict()
    counts = df.value_counts(["mp","ammo","mre","part"])
    for mp, ammo, mre, part in counts[counts>=500].keys():
        sub_df = df.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
        if len(sub_df)==0:
            continue

        cnt = len(sub_df)
        record = {}
        record["total"] = cnt
        record["rank5"] = sum(sub_df["equip_rank"]==5)/cnt
        for eid in filter(lambda x: gamedata["equip"][x]["rank"]==5, set(sub_df["equip_id"])):
            record[gamedata["equip"][eid]["name"]] = sum(sub_df["equip_id"]==eid)/cnt
        for k, v in record.items():
            series[((mp,ammo,mre,part),k)] = v
    series = pd.Series(series)
    series.index.set_names(["formula","target"],inplace=True)
    return series

result = []
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    ret = analyze(sub_df)
    ret.name = name
    result.append(ret)

result = pd.concat(result, axis=1).fillna(value='')
result.sort_index(kind="stable", key=lambda col: col.map(lambda x:0) if col.name=="target" else col.map(lambda x: (x[0]+x[1]+x[2]+x[3]*3, x)), inplace=True)
result.to_excel("formula.xlsx", merge_cells=False)

In [36]:
sub_df = df.query({k:v for k,v in eventrecord["equip"]}['日常（30200）'])
formulas = sub_df.value_counts(["mp","ammo","mre","part"])[:25].keys()

In [39]:

result = []
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    ret = analyze(sub_df)
    ret.name = name
    result.append(ret)

result = pd.concat(result, axis=1).fillna(0)
result.sort_index(kind="stable", key=lambda col: col.map(lambda x:0) if col.name=="..." else col, inplace=True)
result.to_excel("formula.xlsx", merge_cells=False)

In [38]:
formulas

MultiIndex([( 10,  10,  10,  10),
            ( 70,  10, 150,  30),
            (140,  10, 110,  10),
            (151, 151, 151, 151),
            (170, 120,  50,  50),
            (150, 150, 150, 150),
            (180, 180,  10,  50),
            (100,  80,  10, 100),
            ( 30, 150,  30, 130),
            ( 10, 150,  90, 100),
            ( 10, 230, 120,  80),
            (150,  50,  50,  50),
            ( 10, 300,  10,  10),
            (100, 250, 150, 170),
            (180,  20, 200,  20),
            (180,  65,  65,  65),
            ( 30,  30,  30, 200),
            ( 80, 220,  80,  80),
            ( 50,  50, 100, 100),
            (300,  10,  10, 200),
            (300, 300, 300, 300),
            ( 10,  10,  10, 300),
            (110, 110, 110, 110),
            (210, 210, 210, 210),
            ( 50, 250, 150, 100)],
           names=['mp', 'ammo', 'mre', 'part'])